# 1. Creación de Escenarios y Particiones por Consulta Clínica para cada día del Set de Pruebas.

## 1.1 Importación de Librerías y Carga del Set de Pruebas heredado de la Fase 1 - Predicción de No Shows.

In [1]:
import pandas as pd
from datetime import datetime
import random

from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
# Ruta del archivo CSV con los datos del Set de Pruebas
file_path = "./test_set_P1-ALL-ADASYN con Prob_No_Show_TabNet.xlsx"

# Leer el archivo EXCEL
test_set_full = pd.read_excel(file_path)

# Nos quedamos con el contenido que necesitamos: 'PatientId', 'ScheduledDay', 'AppointmentDay', 'NoShow', 'Prob_NoShow'
test_set = test_set_full[['PatientId', 'ScheduledDay', 'AppointmentDay', 'NoShow', 'Prob_NoShow']]

# Convertimos 'ScheduledDay' y 'AppointmentDay' a datetime
pd.options.mode.copy_on_write = True
test_set['ScheduledDay'] = pd.to_datetime(test_set['ScheduledDay'])
test_set['AppointmentDay'] = pd.to_datetime(test_set['AppointmentDay'])

# Mostrar el contenido del DataFrame
test_set

,PatientId,ScheduledDay,AppointmentDay,NoShow,Prob_NoShow
0,1216586867796,2015-12-07 10:40:59,2016-06-03,True,0.564009
1,31899595421534,2015-12-07 10:42:42,2016-06-03,False,0.536110
2,9582232334148,2015-12-07 10:43:01,2016-06-03,False,0.393693
3,3516253533716,2015-12-07 10:43:17,2016-06-03,False,0.371280
4,454287126844,2015-12-07 10:43:34,2016-06-03,False,0.446127
...,...,...,...,...,...
22092,729255235141745,2016-06-08 19:32:25,2016-06-08,False,0.030953
22093,947614361749238,2016-06-08 19:32:56,2016-06-08,False,0.125537
22094,356247857784,2016-06-08 19:33:23,2016-06-08,False,0.045798
22095,234131759175,2016-06-08 19:58:52,2016-06-08,False,0.045798


In [3]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22097 entries, 0 to 22096
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   PatientId       22097 non-null  int64         
 1   ScheduledDay    22097 non-null  datetime64[ns]
 2   AppointmentDay  22097 non-null  datetime64[ns]
 3   NoShow          22097 non-null  bool          
 4   Prob_NoShow     22097 non-null  float64       
dtypes: bool(1), datetime64[ns](2), float64(1), int64(1)
memory usage: 712.2 KB


## 1.2 Creación de Set de Entrenamiento y Set de Validación.

Reservamos un 20% de los datos para verificar la Optimización en la Programación de Citas Médicas una vez entrenado el modelo.

In [4]:
# Calculate the total number of NoShows
total_no_shows = test_set['NoShow'].sum()

# Calculate the number of NoShows for each dataset
no_shows_80_percent = int(total_no_shows * 0.8)
no_shows_20_percent = total_no_shows - no_shows_80_percent

# Split the dataset into two based on the calculated proportions
train_set = test_set[test_set['NoShow'] == 0].sample(frac = 0.8, random_state = 42)
train_set = pd.concat([
        train_set,
        test_set[test_set['NoShow'] == 1].sample(n = no_shows_80_percent, random_state = 42)
])

val_set = test_set.drop(train_set.index)

# Verify the proportions of NoShows in each dataset
proportion_NoShows_train_set = train_set['NoShow'].mean()
proportion_NoShows_val_set = val_set['NoShow'].mean()

print(f'Proporción de NoShows en el Set de Entrenamiento: {proportion_NoShows_train_set:.4f}')
print(f'Proporción de NoShows en el Set de Validación: {proportion_NoShows_val_set:.4f}')

Proporción de NoShows en el Set de Entrenamiento: 0.2018
Proporción de NoShows en el Set de Validación: 0.2020


## 1.3 Separación de datos por 'AppointmentDay'.

Agrupamos los datos por 'AppointmentDay', revisamos y guardamos dichos subconjuntos. Aunque esto sólo será de aplicación cuando entrenemos el algoritmo de ML que calcule los mejores spots para generar el overbooking.

In [5]:
# Group the DataFrame by 'AppointmentDay'
grouped_train_data = train_set.groupby('AppointmentDay')
grouped_val_data = val_set.groupby('AppointmentDay')

# Create a dictionary to store the smaller datasets
train_datasets = {}
val_datasets = {}

# Iterate over each group and store the corresponding dataset in the dictionary
for day, group in grouped_train_data:
    train_datasets[day] = group
for day, group in grouped_val_data:
    val_datasets[day] = group

# Access, review and store the datasets for specific days
train_set_May30 = train_datasets[datetime(2016, 5, 30)]
print(f"El set de entrenamiento para el 30 de mayo de 2016 tiene \
{train_set_May30.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_May30['NoShow'].mean():.2%}.")
val_set_May30 = val_datasets[datetime(2016, 5, 30)]
print(f"El set de validación para el 30 de mayo de 2016 tiene \
{val_set_May30.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_May30['NoShow'].mean():.2%}.")
print("\033[1mDescartamos los datos del 30 de mayo de 2016 por \
insuficiencia y por ser todos No Show.\033[0m\n")

train_set_May31 = train_datasets[datetime(2016, 5, 31)]
print(f"El set de entrenamiento para el 31 de mayo de 2016 tiene \
{train_set_May31.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_May31['NoShow'].mean():.2%}.")
val_set_May31 = val_datasets[datetime(2016, 5, 31)]
print(f"El set de validación para el 31 de mayo de 2016 tiene \
{val_set_May31.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_May31['NoShow'].mean():.2%}.")
print("\033[1mDescartamos los datos del 31 de mayo de 2016 porque \
el alto número de No Shows desvirtuan la programación y cálculo de los costes.\033[0m\n")

train_set_June01 = train_datasets[datetime(2016, 6, 1)]
print(f"El set de entrenamiento para el 1 de junio de 2016 tiene \
{train_set_June01.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June01['NoShow'].mean():.2%}.")
val_set_June01 = val_datasets[datetime(2016, 6, 1)]
print(f"El set de validación para el 1 de junio de 2016 tiene \
{val_set_June01.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June01['NoShow'].mean():.2%}.\n")

train_set_June02 = train_datasets[datetime(2016, 6, 2)]
print(f"El set de entrenamiento para el 2 de junio de 2016 tiene \
{train_set_June02.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June02['NoShow'].mean():.2%}.")
val_set_June02 = val_datasets[datetime(2016, 6, 2)]
print(f"El set de validación para el 2 de junio de 2016 tiene \
{val_set_June02.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June02['NoShow'].mean():.2%}.\n")

train_set_June03 = train_datasets[datetime(2016, 6, 3)]
print(f"El set de entrenamiento para el 3 de junio de 2016 tiene \
{train_set_June03.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June03['NoShow'].mean():.2%}.")
val_set_June03 = val_datasets[datetime(2016, 6, 3)]
print(f"El set de validación para el 3 de junio de 2016 tiene \
{val_set_June03.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June03['NoShow'].mean():.2%}.\n")

train_set_June06 = train_datasets[datetime(2016, 6, 6)]
print(f"El set de entrenamiento para el 6 de junio de 2016 tiene \
{train_set_June06.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June06['NoShow'].mean():.2%}.")
val_set_June06 = val_datasets[datetime(2016, 6, 6)]
print(f"El set de validación para el 6 de junio de 2016 tiene \
{val_set_June06.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June06['NoShow'].mean():.2%}.\n")

train_set_June07 = train_datasets[datetime(2016, 6, 7)]
print(f"El set de entrenamiento para el 7 de junio de 2016 tiene \
{train_set_June07.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June07['NoShow'].mean():.2%}.")
val_set_June07 = val_datasets[datetime(2016, 6, 7)]
print(f"El set de validación para el 7 de junio de 2016 tiene \
{val_set_June07.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June07['NoShow'].mean():.2%}.\n")

train_set_June08 = train_datasets[datetime(2016, 6, 8)]
print(f"El set de entrenamiento para el 8 de junio de 2016 tiene \
{train_set_June08.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June08['NoShow'].mean():.2%}.")
val_set_June08 = val_datasets[datetime(2016, 6, 8)]
print(f"El set de validación para el 8 de junio de 2016 tiene \
{val_set_June08.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June08['NoShow'].mean():.2%}.\n")

print("\033[1mREHACEMOS TODO EL PROCESO DESCARTANDO LOS DATOS DEL 30 Y 31 DE MAYO.\033[0m\n")

El set de entrenamiento para el 30 de mayo de 2016 tiene 45 citas, con un porcentaje de NoShows del 100.00%.
El set de validación para el 30 de mayo de 2016 tiene 15 citas, con un porcentaje de NoShows del 100.00%.
Descartamos los datos del 30 de mayo de 2016 por insuficiencia y por ser todos No Show.

El set de entrenamiento para el 31 de mayo de 2016 tiene 613 citas, con un porcentaje de NoShows del 66.72%.
El set de validación para el 31 de mayo de 2016 tiene 152 citas, con un porcentaje de NoShows del 67.11%.
Descartamos los datos del 31 de mayo de 2016 porque el alto número de No Shows desvirtuan la programación y cálculo de los costes.

El set de entrenamiento para el 1 de junio de 2016 tiene 2472 citas, con un porcentaje de NoShows del 18.16%.
El set de validación para el 1 de junio de 2016 tiene 589 citas, con un porcentaje de NoShows del 21.22%.

El set de entrenamiento para el 2 de junio de 2016 tiene 2582 citas, con un porcentaje de NoShows del 18.20%.
El set de validación p

In [6]:
# Quitamos las citas del 30 y 31 de mayo.
new_test_set = test_set[~test_set['AppointmentDay'].dt.strftime('%Y-%m-%d').isin(['2016-05-30', '2016-05-31'])]

print(new_test_set[new_test_set['AppointmentDay'] == datetime(2016, 5, 30)])
print(new_test_set[new_test_set['AppointmentDay'] == datetime(2016, 5, 31)])
new_test_set

Empty DataFrame
Columns: [PatientId, ScheduledDay, AppointmentDay, NoShow, Prob_NoShow]
Index: []
Empty DataFrame
Columns: [PatientId, ScheduledDay, AppointmentDay, NoShow, Prob_NoShow]
Index: []


,PatientId,ScheduledDay,AppointmentDay,NoShow,Prob_NoShow
0,1216586867796,2015-12-07 10:40:59,2016-06-03,True,0.564009
1,31899595421534,2015-12-07 10:42:42,2016-06-03,False,0.536110
2,9582232334148,2015-12-07 10:43:01,2016-06-03,False,0.393693
3,3516253533716,2015-12-07 10:43:17,2016-06-03,False,0.371280
4,454287126844,2015-12-07 10:43:34,2016-06-03,False,0.446127
...,...,...,...,...,...
22092,729255235141745,2016-06-08 19:32:25,2016-06-08,False,0.030953
22093,947614361749238,2016-06-08 19:32:56,2016-06-08,False,0.125537
22094,356247857784,2016-06-08 19:33:23,2016-06-08,False,0.045798
22095,234131759175,2016-06-08 19:58:52,2016-06-08,False,0.045798


In [7]:
new_test_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21272 entries, 0 to 22096
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   PatientId       21272 non-null  int64         
 1   ScheduledDay    21272 non-null  datetime64[ns]
 2   AppointmentDay  21272 non-null  datetime64[ns]
 3   NoShow          21272 non-null  bool          
 4   Prob_NoShow     21272 non-null  float64       
dtypes: bool(1), datetime64[ns](2), float64(1), int64(1)
memory usage: 851.7 KB


In [8]:
# Calculate the total number of NoShows
total_no_shows = new_test_set['NoShow'].sum()

# Calculate the number of NoShows for each dataset
no_shows_80_percent = int(total_no_shows * 0.8)
no_shows_20_percent = total_no_shows - no_shows_80_percent

# Split the dataset into two based on the calculated proportions
train_set = new_test_set[new_test_set['NoShow'] == 0].sample(frac = 0.8, random_state = 42)
train_set = pd.concat([train_set,
        new_test_set[new_test_set['NoShow'] == 1].sample(n = no_shows_80_percent, random_state = 42)])

val_set = new_test_set.drop(train_set.index)

# Verify the proportions of NoShows in each dataset
proportion_NoShows_train_set = train_set['NoShow'].mean()
proportion_NoShows_val_set = val_set['NoShow'].mean()

print(f'Proporción de NoShows en el Set de Entrenamiento: {proportion_NoShows_train_set:.4f}')
print(f'Proporción de NoShows en el Set de Validación: {proportion_NoShows_val_set:.4f}')

Proporción de NoShows en el Set de Entrenamiento: 0.1829
Proporción de NoShows en el Set de Validación: 0.1829


In [9]:
# Group the DataFrame by 'AppointmentDay'
grouped_total_data = new_test_set.groupby('AppointmentDay')
grouped_train_data = train_set.groupby('AppointmentDay')
grouped_val_data = val_set.groupby('AppointmentDay')

# Create a dictionary to store the smaller datasets
total_datasets = {}
train_datasets = {}
val_datasets = {}

# Iterate over each group and store the corresponding dataset in the dictionary
for day, group in grouped_total_data:
    total_datasets[day] = group
for day, group in grouped_train_data:
    train_datasets[day] = group
for day, group in grouped_val_data:
    val_datasets[day] = group

# Access, review and store the datasets for specific days
total_set_June01 = total_datasets[datetime(2016, 6, 1)]
print(f"El set con todas las citas médicas para el 1 de junio de 2016 tiene \
{total_set_June01.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June01['NoShow'].mean():.2%}.")
train_set_June01 = train_datasets[datetime(2016, 6, 1)]
print(f"El set de entrenamiento para el 1 de junio de 2016 tiene \
{train_set_June01.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June01['NoShow'].mean():.2%}.")
val_set_June01 = val_datasets[datetime(2016, 6, 1)]
print(f"El set de validación para el 1 de junio de 2016 tiene \
{val_set_June01.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June01['NoShow'].mean():.2%}.\n")

total_set_June02 = total_datasets[datetime(2016, 6, 2)]
print(f"El set con todas las citas médicas para el 2 de junio de 2016 tiene \
{total_set_June02.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June02['NoShow'].mean():.2%}.")
train_set_June02 = train_datasets[datetime(2016, 6, 2)]
print(f"El set de entrenamiento para el 2 de junio de 2016 tiene \
{train_set_June02.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June02['NoShow'].mean():.2%}.")
val_set_June02 = val_datasets[datetime(2016, 6, 2)]
print(f"El set de validación para el 2 de junio de 2016 tiene \
{val_set_June02.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June02['NoShow'].mean():.2%}.\n")

total_set_June03 = total_datasets[datetime(2016, 6, 3)]
print(f"El set con todas las citas médicas para el 3 de junio de 2016 tiene \
{total_set_June03.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June03['NoShow'].mean():.2%}.")
train_set_June03 = train_datasets[datetime(2016, 6, 3)]
print(f"El set de entrenamiento para el 3 de junio de 2016 tiene \
{train_set_June03.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June03['NoShow'].mean():.2%}.")
val_set_June03 = val_datasets[datetime(2016, 6, 3)]
print(f"El set de validación para el 3 de junio de 2016 tiene \
{val_set_June03.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June03['NoShow'].mean():.2%}.\n")

total_set_June06 = total_datasets[datetime(2016, 6, 6)]
print(f"El set con todas las citas médicas para el 6 de junio de 2016 tiene \
{total_set_June06.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June06['NoShow'].mean():.2%}.")
train_set_June06 = train_datasets[datetime(2016, 6, 6)]
print(f"El set de entrenamiento para el 6 de junio de 2016 tiene \
{train_set_June06.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June06['NoShow'].mean():.2%}.")
val_set_June06 = val_datasets[datetime(2016, 6, 6)]
print(f"El set de validación para el 6 de junio de 2016 tiene \
{val_set_June06.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June06['NoShow'].mean():.2%}.\n")

total_set_June07 = total_datasets[datetime(2016, 6, 7)]
print(f"El set con todas las citas médicas para el 7 de junio de 2016 tiene \
{total_set_June07.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June07['NoShow'].mean():.2%}.")
train_set_June07 = train_datasets[datetime(2016, 6, 7)]
print(f"El set de entrenamiento para el 7 de junio de 2016 tiene \
{train_set_June07.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June07['NoShow'].mean():.2%}.")
val_set_June07 = val_datasets[datetime(2016, 6, 7)]
print(f"El set de validación para el 7 de junio de 2016 tiene \
{val_set_June07.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June07['NoShow'].mean():.2%}.\n")

total_set_June08 = total_datasets[datetime(2016, 6, 8)]
print(f"El set con todas las citas médicas para el 8 de junio de 2016 tiene \
{total_set_June08.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June08['NoShow'].mean():.2%}.")
train_set_June08 = train_datasets[datetime(2016, 6, 8)]
print(f"El set de entrenamiento para el 8 de junio de 2016 tiene \
{train_set_June08.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June08['NoShow'].mean():.2%}.")
val_set_June08 = val_datasets[datetime(2016, 6, 8)]
print(f"El set de validación para el 8 de junio de 2016 tiene \
{val_set_June08.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June08['NoShow'].mean():.2%}.\n")

El set con todas las citas médicas para el 1 de junio de 2016 tiene 3061 citas, con un porcentaje de NoShows del 18.75%.
El set de entrenamiento para el 1 de junio de 2016 tiene 2466 citas, con un porcentaje de NoShows del 19.02%.
El set de validación para el 1 de junio de 2016 tiene 595 citas, con un porcentaje de NoShows del 17.65%.

El set con todas las citas médicas para el 2 de junio de 2016 tiene 3248 citas, con un porcentaje de NoShows del 18.29%.
El set de entrenamiento para el 2 de junio de 2016 tiene 2599 citas, con un porcentaje de NoShows del 17.89%.
El set de validación para el 2 de junio de 2016 tiene 649 citas, con un porcentaje de NoShows del 19.88%.

El set con todas las citas médicas para el 3 de junio de 2016 tiene 3163 citas, con un porcentaje de NoShows del 19.60%.
El set de entrenamiento para el 3 de junio de 2016 tiene 2565 citas, con un porcentaje de NoShows del 19.69%.
El set de validación para el 3 de junio de 2016 tiene 598 citas, con un porcentaje de NoShows

# 2. Cálculo de Costes

## 2.1 Regla de Asignación de Cita
### **Por Probabilidades de Asistencia**

Al primer paciente que pide una cita para un determinado día se le asigna al primer slot de la primera consulta, y se guarda también su probabilidad de asistencia.

Cuando un nuevo paciente solicita cita para el mismo día, se revisa slot por slot, y se le coloca en el primer slot donde la suma de las probabilidades de asistencia de todos los pacientes asignados a dicho slot, incluido este último, sea menor de un porcentaje determinado.

Si no "cabe" en ningún slot, se "abre" otra consulta.

In [10]:
def asignar_paciente(agenda_medica: dict, paciente_id: str, fecha: datetime.date, prob_show: float):
    paciente_sin_asignar = True

    while paciente_sin_asignar:
        asignado = False

        # Iteramos sobre consultas y slots para asignar el paciente
        for consulta_key, slots in agenda_medica[fecha].items():
            for slot_key, pacientes in slots.items():
                suma_prob_show = sum(paciente[1] for paciente in pacientes)
                for paciente in pacientes:
                    suma_prob_show += paciente[1]
                
                # Verificamos si el paciente puede ser asignado al slot
                if len(pacientes) < 3 and suma_prob_show + prob_show < LISTON_PROB_SHOW:
                    agenda_medica[fecha][consulta_key][slot_key].append((paciente_id, prob_show))
                    print(f'\t{fecha} {consulta_key} {slot_key}: {agenda_medica[fecha][consulta_key][slot_key]}\n')
                    asignado = True
                    break  # Salimos del bucle de slots

            if asignado:
                break  # Salimos del bucle de consultas
        
        if asignado:
            paciente_sin_asignar = False
        else:
            # Si no se pudo asignar el paciente a ningún slot existente, abrimos un nuevo slot en una consulta existente
            for consulta_key, slots in agenda_medica[fecha].items():
                if len(slots) < 24:
                    num_slot = len(slots) + 1
                    slot_key = f'slot_{num_slot:02d}'
                    agenda_medica[fecha][consulta_key][slot_key] = []
                    print(f'Abrimos {slot_key} en {consulta_key} para el {fecha}.\n')
                    agenda_medica[fecha][consulta_key][slot_key].append((paciente_id, prob_show))
                    print(f'\t{fecha} {consulta_key} {slot_key}: {agenda_medica[fecha][consulta_key][slot_key]}\n')
                    paciente_sin_asignar = False
                    break  # Salimos del bucle de consultas

            if paciente_sin_asignar:
                # Si no hemos podido asignar (porque todas las consultas están a tope), abre una nueva consulta y un nuevo slot
                num_consulta = len(agenda_medica[fecha]) + 1
                consulta_key = f'consulta_{num_consulta:03d}'
                agenda_medica[fecha][consulta_key] = {}
                print(f'Abrimos {consulta_key} para el {fecha}.')
                agenda_medica[fecha][consulta_key]['slot_01'] = []
                print(f'Abrimos slot_01 en {consulta_key} para el {fecha}.\n')
                agenda_medica[fecha][consulta_key]['slot_01'].append((paciente_id, prob_show))
                print(f'\t{fecha} {consulta_key} {slot_key}: {agenda_medica[fecha][consulta_key]["slot_01"]}\n')
                paciente_sin_asignar = False  # Rompemos el while_condition

    return agenda_medica

In [30]:
final_total_test_set = new_test_set.sort_values('ScheduledDay')

# Establecemos el límite de probabilidad de Show hasta donde seguimos asignando pacientes al mismo slot
LISTON_PROB_SHOW = 0.9

agenda_medica = {}

for cita in final_total_test_set.itertuples(index=True, name='Pandas'):
    fecha_cita = cita.AppointmentDay.date()
    prob_show = 1 - cita.Prob_NoShow

    if fecha_cita in agenda_medica.keys():
        agenda_medica = asignar_paciente(agenda_medica, cita.PatientId, fecha_cita, prob_show)
    else:
        agenda_medica[fecha_cita] = {}
        print(f'Abrimos agenda para el {fecha_cita}.')
        agenda_medica[fecha_cita]['consulta_001'] = {}
        print(f'Abrimos consulta_001 para el {fecha_cita}.')
        agenda_medica[fecha_cita]['consulta_001']['slot_01'] = []
        print(f'Abrimos slot_01 en consulta_001 para el {fecha_cita}.\n')
        agenda_medica[fecha_cita]['consulta_001']['slot_01'].append((cita.PatientId, prob_show))
        print(f'\t{fecha_cita} consulta_001 slot_01: {agenda_medica[fecha_cita]["consulta_001"]["slot_01"]}\n')

Abrimos agenda para el 2016-06-03.
Abrimos consulta_001 para el 2016-06-03.
Abrimos slot_01 en consulta_001 para el 2016-06-03.

	2016-06-03 consulta_001 slot_01: [(1216586867796, 0.43599135)]

Abrimos slot_02 en consulta_001 para el 2016-06-03.

	2016-06-03 consulta_001 slot_02: [(31899595421534, 0.46389024999999995)]

Abrimos slot_03 en consulta_001 para el 2016-06-03.

	2016-06-03 consulta_001 slot_03: [(9582232334148, 0.60630724)]

Abrimos slot_04 en consulta_001 para el 2016-06-03.

	2016-06-03 consulta_001 slot_04: [(3516253533716, 0.62872013)]

Abrimos slot_05 en consulta_001 para el 2016-06-03.

	2016-06-03 consulta_001 slot_05: [(454287126844, 0.55387294)]

Abrimos slot_06 en consulta_001 para el 2016-06-03.

	2016-06-03 consulta_001 slot_06: [(941625887116382, 0.60102943)]

Abrimos slot_07 en consulta_001 para el 2016-06-03.

	2016-06-03 consulta_001 slot_07: [(351265918724495, 0.5669160200000001)]

Abrimos slot_08 en consulta_001 para el 2016-06-03.

	2016-06-03 consulta_001

In [31]:
# Convertirmos el diccionario agenda medica en una lista de diccionarios aplanados
agenda_medica_aplanada = []
for fecha_key, consultas in agenda_medica.items():
    for consulta_key, slots in consultas.items():
        for slot_key, pacientes in slots.items():
            for paciente_id, prob_show in pacientes:
                agenda_medica_aplanada.append({
                    'Fecha': fecha_key,
                    'Consulta': consulta_key,
                    'Slot': slot_key,
                    'Paciente_ID': paciente_id,
                    'Probabilidad_Show': prob_show
                })

# Creamos un DataFrame a partir de la lista de diccionarios
df_agenda_medica = pd.DataFrame(agenda_medica_aplanada)

# Ordenamos la Agenda Médica por Fecha, Número de Consulta y Número de Slot
df_agenda_medica = df_agenda_medica.sort_values(by=['Fecha', 'Consulta', 'Slot'], ascending=[True, True, True])

# Mostrar el DataFrame
df_agenda_medica.to_excel('Agenda Medica.xlsx', index=False)

In [32]:
# Revisamos número de consultas por Fecha
consultas_por_fecha = []
for fecha in agenda_medica.keys():
    consultas_por_fecha.append({
        'Fecha': fecha,
        'Número Consultas': len(agenda_medica[fecha])
})

df_consultas_por_fecha = pd.DataFrame(consultas_por_fecha)

# Revisamos número de pacientes por Consulta
pacientes_por_consulta = {}
for fecha_key, consultas in agenda_medica.items():
    pacientes_por_consulta[fecha_key] = {}
    for consulta_key, slots in consultas.items():
        num_pacientes_por_consulta = 0
        for pacientes in slots.values(): 
            num_pacientes_por_consulta += len(pacientes)
        pacientes_por_consulta[fecha_key][consulta_key] = num_pacientes_por_consulta

pacientes_por_consulta

{datetime.date(2016, 6, 3): {'consulta_001': 29,
  'consulta_002': 30,
  'consulta_003': 27,
  'consulta_004': 31,
  'consulta_005': 32,
  'consulta_006': 32,
  'consulta_007': 33,
  'consulta_008': 39,
  'consulta_009': 30,
  'consulta_010': 34,
  'consulta_011': 31,
  'consulta_012': 29,
  'consulta_013': 35,
  'consulta_014': 28,
  'consulta_015': 31,
  'consulta_016': 32,
  'consulta_017': 29,
  'consulta_018': 28,
  'consulta_019': 25,
  'consulta_020': 30,
  'consulta_021': 27,
  'consulta_022': 28,
  'consulta_023': 26,
  'consulta_024': 26,
  'consulta_025': 25,
  'consulta_026': 30,
  'consulta_027': 27,
  'consulta_028': 25,
  'consulta_029': 30,
  'consulta_030': 26,
  'consulta_031': 27,
  'consulta_032': 27,
  'consulta_033': 26,
  'consulta_034': 26,
  'consulta_035': 26,
  'consulta_036': 27,
  'consulta_037': 26,
  'consulta_038': 28,
  'consulta_039': 25,
  'consulta_040': 24,
  'consulta_041': 26,
  'consulta_042': 24,
  'consulta_043': 25,
  'consulta_044': 25,
  'co

In [33]:
# Calculamos la media de pacientes por consulta para cada fecha
media_pacientes_por_consulta = {}
for fecha, consultas in pacientes_por_consulta.items():
    total_pacientes = sum(consultas.values())
    num_consultas = len(consultas)
    media_pacientes_por_consulta[fecha] = total_pacientes / num_consultas

# Añadimos la columna de media de pacientes por consulta al DataFrame
df_consultas_por_fecha['Media Pacientes por Consulta'] = df_consultas_por_fecha['Fecha'].map(media_pacientes_por_consulta)

# Ordenamos la Agenda Médica por Fecha, Número de Consulta y Número de Slot
df_consultas_por_fecha = df_consultas_por_fecha.sort_values(by=['Fecha'], ascending=[True])

df_consultas_por_fecha

,Fecha,Número Consultas,Media Pacientes por Consulta
1,2016-06-01,121,25.297521
4,2016-06-02,128,25.375000
0,2016-06-03,123,25.715447
3,2016-06-06,149,25.516779
5,2016-06-07,153,25.385621
2,2016-06-08,164,25.085366


### 2.1.1 Asignaciones de cita

In [13]:
NUM_SLOTS_PER_DAY = 24

# 01 Junio 2016:
# 10 settings x 127 consultas = 1270 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June01_R11 = {}
for i, setting in enumerate(total_groups_June01_24p.keys()):
    for j, group in enumerate(total_groups_June01_24p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June01_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June01_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

# 02 Junio 2016:
# 10 settings x 135 consultas = 1350 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June02_R11 = {}
for i, setting in enumerate(total_groups_June02_24p.keys()):
    for j, group in enumerate(total_groups_June02_24p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June02_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June02_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

# 03 Junio 2016:
# 10 settings x 131 consultas = 1310 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June03_R11 = {}
for i, setting in enumerate(total_groups_June03_24p.keys()):
    for j, group in enumerate(total_groups_June03_24p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June03_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June03_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

# 06 Junio 2016:
# 10 settings x 158 consultas = 1580 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June06_R11 = {}
for i, setting in enumerate(total_groups_June06_24p.keys()):
    for j, group in enumerate(total_groups_June06_24p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June06_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June06_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

# 07 Junio 2016:
# 10 settings x 161 consultas = 1610 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June07_R11 = {}
for i, setting in enumerate(total_groups_June07_24p.keys()):
    for j, group in enumerate(total_groups_June07_24p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June07_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June07_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

# 08 Junio 2016:
# 10 settings x 171 consultas = 1710 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June08_R11 = {}
for i, setting in enumerate(total_groups_June08_24p.keys()):
    for j, group in enumerate(total_groups_June08_24p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June08_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June08_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

### 2.1.2 Cálculo de Costes: Idle Time, Waiting Time y Over Time

In [14]:
def calculo_costes(set:pd.DataFrame, groups: dict, slot_apps: dict):
    NUM_SETTINGS = len(groups)
    NUM_DOCTORS = len(groups['setting 1'])
    print(f'Número de consultas: {len(slot_apps)}  \
=  {NUM_SETTINGS} settings diferentes * {NUM_DOCTORS} doctores pasando consulta.')
    
    copied_slot_apps = slot_apps.copy()   # Hacemos una copia para no sobre escribir fuera

    idle_costs = {}   # Diccionario de Costos de inactividad del doctor
    waiting_costs = {}   # Diccionario de Costos de espera del paciente
    overtime_costs = {}   # Diccionario de Costos de tiempo extra del doctor

    for setting in range(1, NUM_SETTINGS+1):
        for consulta in range(1, NUM_DOCTORS+1):
            scenery = f'slot_apps_{setting}_{consulta}'
            # Parámetros iniciales
            idle_costs[scenery] = 24   # Coste máximo, no se ha pasado ninguna consulta.
            waiting_costs[scenery] = 0   # Coste mínimo, nadie ha esperado.
            overtime_costs[scenery] = 0   # Coste mínimo, sin trabajo extra.

            patients_waiting_at_13 = 0   # Pacientes esperando a medio día
            patients_waiting_at_18 = 0   # Pacientes esperanto por la tarde
            slot = 1  # Primer Slot
            while slot < 25:
                slot_ocupado = False   # Inicialmente
                idx_insertion = 0
                for patient in slot_apps[scenery][slot]:
                    patient_row_index = set[set['PatientId'] == patient].index
                    if slot_ocupado and set.loc[patient_row_index, 'NoShow'].any() == False:
                        if slot == 12:
                            patients_waiting_at_13 += 1
                            if patients_waiting_at_13 > 3:
                                print(f'¡OJO! El doctor no tiene el tiempo para \
atender a este paciente {patient} de 13:00 a 14:00.\nSe mueve al slot de las 14:00.')
                                waiting_costs[scenery] += patients_waiting_at_13 * 1
                                copied_slot_apps[scenery][slot+1].insert(idx_insertion, patient)
                                idx_insertion += 1
                            else:
                                overtime_costs[scenery] += 1
                                waiting_costs[scenery] += patients_waiting_at_13 * 1
                        elif slot == 24:
                            patients_waiting_at_18 += 1
                            overtime_costs[scenery] += 1
                            waiting_costs[scenery] += patients_waiting_at_18 * 1
                        else:
                            waiting_costs[scenery] += 1
                            copied_slot_apps[scenery][slot+1].insert(idx_insertion, patient)
                            idx_insertion += 1
                    elif set.loc[patient_row_index, 'NoShow'].any() == False:
                        slot_ocupado = True
                        idle_costs[scenery] -= 1
                slot += 1

    average_idle_cost = sum(list(idle_costs.values())) / len(slot_apps)
    print(f'La media de los Costes de Inactividad en los {NUM_SETTINGS} escenarios distintos \
para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_idle_cost:.2f}.')

    average_waiting_cost = sum(list(waiting_costs.values())) / len(slot_apps)
    print(f'La media de los Costes de Espera de los Pacientes en los {NUM_SETTINGS} escenarios distintos \
para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_waiting_cost:.2f}.')

    average_overtime_cost = sum(list(overtime_costs.values())) / len(slot_apps)
    print(f'La media de los Costes de Tiempo Extra en los {NUM_SETTINGS} escenarios distintos \
para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_overtime_cost:.2f}.')
    
    costs = {'idle_costs': idle_costs,
            'waiting_costs': waiting_costs,
            'overtime_costs': overtime_costs}
    average_costs = {'average_idle_cost': average_idle_cost,
                    'average_waiting_cost': average_waiting_cost,
                    'average_overtime_cost': average_overtime_cost}
    
    return costs, average_costs

#### **Misma función con todos los prints, para entender el código y supervisar los cálculos cuando se ejecuta.**

_(Recomiendo correrlo sólo con un día para no bloquear el equipo)_

    def calculo_costes(set:pd.DataFrame, groups: dict, slot_apps: dict):
        NUM_SETTINGS = len(groups)
        NUM_DOCTORS = len(groups['setting 1'])
        print(f'Número de consultas: {len(slot_apps)}  \
    =  {NUM_SETTINGS} settings diferentes * {NUM_DOCTORS} doctores pasando consulta.')
        
        copied_slot_apps = slot_apps.copy()   # Hacemos una copia para no sobre escribir fuera
    
        idle_costs = {}   # Costos de inactividad del doctor
        waiting_costs = {}   # Costos de espera del paciente
        overtime_costs = {}   # Costos de tiempo extra del doctor
    
        for setting in range(1, NUM_SETTINGS+1):
            for consulta in range(1, NUM_DOCTORS+1):
                scenery = f'slot_apps_{setting}_{consulta}'
                print('\n', scenery)
                # Parámetros iniciales
                idle_costs[scenery] = 24   # Coste máximo, no se ha pasado ninguna consulta.
                print(f'Idle[{scenery}] -> {idle_costs[scenery]}')
                waiting_costs[scenery] = 0   # Coste mínimo, nadie ha esperado.
                print(f'Waiting[{scenery}] -> {waiting_costs[scenery]}')
                overtime_costs[scenery] = 0   # Coste mínimo, sin trabajo extra.
                print(f'Overtime[{scenery}] -> {overtime_costs[scenery]}')
    
                patients_waiting_at_13 = 0   # Pacientes esperando a medio día
                patients_waiting_at_18 = 0   # Pacientes esperanto por la tarde
                slot = 1  # Primer Slot
                print('EMPEZAMOS')
                while slot < 25:
                    slot_ocupado = False   # Inicialmente
                    print(f'Slot {slot}')
                    idx_insertion = 0
                    print(f'\tIdx_insertion = {idx_insertion}')
                    for patient in slot_apps[scenery][slot]:
                        print(f'\tPatient {patient} - Slot_ocupado = {slot_ocupado}')
                        patient_row_index = set[set['PatientId'] == patient].index
                        print(f"\t¿Faltó a la cita? {set.loc[patient_row_index, 'NoShow'].any()}")
                        if slot_ocupado and set.loc[patient_row_index, 'NoShow'].any() == False:
                            if slot == 12:
                                patients_waiting_at_13 += 1
                                print(f'\tPatients waiting at 13:00: {patients_waiting_at_13}')
                                if patients_waiting_at_13 > 3:
                                    print(f'¡OJO! El doctor no tiene el tiempo para \
    atender a este paciente {patient} de 13:00 a 14:00.\nSe mueve al slot de las 14:00.')
                                    waiting_costs[scenery] += patients_waiting_at_13 * 1
                                    copied_slot_apps[scenery][slot+1].insert(idx_insertion, patient)
                                    idx_insertion += 1
                                    print(f'\tIdx_insertion = {idx_insertion}')
                                else:
                                    overtime_costs[scenery] += 1
                                    print(f'\tOvertime_cost[{scenery}] -> {overtime_costs[scenery]}')
                                    waiting_costs[scenery] += patients_waiting_at_13 * 1
                                    print(f'\tWaiting_cost[{scenery}] -> {waiting_costs[scenery]}')
                            elif slot == 24:
                                patients_waiting_at_18 += 1
                                print(f'\tPatients waiting at 18:00: {patients_waiting_at_18}')
                                overtime_costs[scenery] += 1
                                print(f'\tOvertime_cost[{scenery}] -> {overtime_costs[scenery]}')
                                waiting_costs[scenery] += patients_waiting_at_18 * 1
                                print(f'\tWaiting_cost[{scenery}] -> {waiting_costs[scenery]}')
                            else:
                                waiting_costs[scenery] += 1
                                print(f'\tWaiting_cost[{scenery}] -> {waiting_costs[scenery]}')
                                copied_slot_apps[scenery][slot+1].insert(idx_insertion, patient)
                                idx_insertion += 1
                                print(f'\tIdx_insertion = {idx_insertion}')
                        elif set.loc[patient_row_index, 'NoShow'].any() == False:
                            slot_ocupado = True
                            idle_costs[scenery] -= 1
                            print(f'\tIdle_cost[{scenery}] -> {idle_costs[scenery]}')
                    slot += 1
    
        average_idle_cost = sum(list(idle_costs.values())) / len(slot_apps)
        print(f'\nLa media de los Costes de Inactividad en los {NUM_SETTINGS} escenarios distintos \
    para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_idle_cost:.2f}.')
    
        average_waiting_cost = sum(list(waiting_costs.values())) / len(slot_apps)
        print(f'La media de los Costes de Espera de los Pacientes en los {NUM_SETTINGS} escenarios distintos \
    para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_waiting_cost:.2f}.')
    
        average_overtime_cost = sum(list(overtime_costs.values())) / len(slot_apps)
        print(f'La media de los Costes de Tiempo Extra en los {NUM_SETTINGS} escenarios distintos \
    para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_overtime_cost:.2f}.')
        
        costs = {'idle_costs': idle_costs,
                'waiting_costs': waiting_costs,
                'overtime_costs': overtime_costs}
        average_costs = {'average_idle_cost': average_idle_cost,
                        'average_waiting_cost': average_waiting_cost,
                        'average_overtime_cost': average_overtime_cost}
        
        return costs, average_costs

    # 3 Junio 2016:
    print('\033[1m03 de junio:\033[0m')
    print('-----------')
    print('Set Total con todas las Citas Médicas:')
    costs_total_June03_R11, average_costs_total_June03_R11 = calculo_costes(
        set = total_set_June03,
        groups = total_groups_June03_R11,
        slot_apps = slot_apps_total_June03_R11
    )
    print('------------------------------------------------------------------\
    -------------------------------------------------------------------------\n')

    # 3 Junio 2016
    print('\033[1m03 de junio:\033[0m')
    print('-----------')
    print('Set Total con todas las Citas Médicas:')
    costs_total_June03_R1231, average_costs_total_June03_R1231 = calculo_costes(
        set = total_set_June03,
        groups = total_groups_June03_R1231,
        slot_apps = slot_apps_total_June03_R1231
    )
    print('------------------------------------------------------------------\
    -------------------------------------------------------------------------\n')


In [15]:
print('\033[1mHIPÓTESIS: Regla de Asignación de Cita\n \
\t\t 1 Slot : 1 Patient\033[0m\n')

# 1 Junio 2016
print('\033[1m01 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June01_R11, average_costs_total_June01_R11 = calculo_costes(
    set = total_set_June01,
    groups = total_groups_June01_24p,
    slot_apps = slot_apps_total_June01_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 2 Junio 2016
print('\033[1m02 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June02_R11, average_costs_total_June02_R11 = calculo_costes(
    set = total_set_June02,
    groups = total_groups_June02_24p,
    slot_apps = slot_apps_total_June02_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 3 Junio 2016:
print('\033[1m03 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June03_R11, average_costs_total_June03_R11 = calculo_costes(
    set = total_set_June03,
    groups = total_groups_June03_24p,
    slot_apps = slot_apps_total_June03_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 6 Junio 2016
print('\033[1m06 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June06_R11, average_costs_total_June06_R11 = calculo_costes(
    set = total_set_June06,
    groups = total_groups_June06_24p,
    slot_apps = slot_apps_total_June06_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 7 Junio 2016
print('\033[1m07 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June07_R11, average_costs_total_June07_R11 = calculo_costes(
    set = total_set_June07,
    groups = total_groups_June07_24p,
    slot_apps = slot_apps_total_June07_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 8 Junio 2016
print('\033[1m08 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June08_R11, average_costs_total_June08_R11 = calculo_costes(
    set = total_set_June08,
    groups = total_groups_June08_24p,
    slot_apps = slot_apps_total_June08_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')


HIPÓTESIS: Regla de Asignación de Cita
 		 1 Slot : 1 Patient

01 de junio:
-----------
Set Total con todas las Citas Médicas:
Número de consultas: 1270  =  10 settings diferentes * 127 doctores pasando consulta.
La media de los Costes de Inactividad en los 10 escenarios distintos para las consultas de los 127 doctores distintos es de 4.50.
La media de los Costes de Espera de los Pacientes en los 10 escenarios distintos para las consultas de los 127 doctores distintos es de 0.00.
La media de los Costes de Tiempo Extra en los 10 escenarios distintos para las consultas de los 127 doctores distintos es de 0.00.
-------------------------------------------------------------------------------------------------------------------------------------------

02 de junio:
-----------
Set Total con todas las Citas Médicas:
Número de consultas: 1350  =  10 settings diferentes * 135 doctores pasando consulta.
La media de los Costes de Inactividad en los 10 escenarios distintos para las consultas de lo

In [16]:
# Función Exportar el DataFrame a Excel

def exportar_tabla_resumen_a_excel(tabla: pd.DataFrame, regla: str) -> None:
    '''
    Exporta un DataFrame a un archivo de Excel con formato personalizado.

    La función toma un DataFrame y una cadena de texto que representa una Regla de Asignación de Cita.
    Crea un archivo de Excel con el DataFrame, añade un título basado en la regla,
    centra el texto de las celdas, aplica un formato numérico a ciertas columnas,
    ajusta el ancho de las columnas y guarda el archivo con un nombre basado en la regla.

    Parámetros:
    tabla (pd.DataFrame): El DataFrame a exportar.
    regla (str): Una cadena de texto que se utilizará para personalizar el nombre del archivo y el título de la hoja de cálculo.

    Retorno:
    None: La función no retorna ningún valor, pero guarda un archivo de Excel en el directorio actual.

    Ejemplo:
    >>> df = pd.DataFrame({
            'A': [1, 2, 3],
            'B': [4, 5, 6],
            'C': [7.1234, 8.5678, 9.9101]
        })
    >>> exportar_tabla_resumen_a_excel(df, 'Ejemplo')
    El DataFrame ha sido exportado exitosamente a Costes_promedio_Ejemplo.xlsx.
    '''
    # Crear un nuevo workbook
    wb = Workbook()
    ws = wb.active

    # Añadir el título
    ws.merge_cells('A1:F1')
    nombre_tabla = 'Costes Promedio Hipótesis ' + regla
    ws['A1'] = nombre_tabla
    ws['A1'].alignment = Alignment(horizontal = 'center')

    # Añadir el DataFrame a la hoja de cálculo
    for r_idx, row in enumerate(dataframe_to_rows(tabla, index = False, header = True), 2):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row = r_idx, column = c_idx, value = value)
            cell.alignment = Alignment(horizontal='center')
            if isinstance(value, float) and c_idx > 2:
                cell.number_format = '0.0000'

    # Ajustar el ancho de las columnas
    for col in ws.iter_cols(min_row = 2,
                            max_row = ws.max_row,
                            min_col = 1,
                            max_col = ws.max_column):
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            if cell.coordinate in ws.merged_cells:  # no considerar celdas fusionadas
                continue
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    # Guardar el archivo
    nombre_archivo = 'Costes_promedio_' + regla + '.xlsx'
    wb.save(nombre_archivo)

    # Mostrar mensaje de confirmación
    print(f'El DataFrame ha sido exportado exitosamente a {nombre_archivo}.')
    
    return None

In [21]:
# Crear la Tabla RESUMEN con los costes medios para esta Regla de Asignación de Citas R11
data_R11 = {
    'Fecha': [
        '2016-06-01', '2016-06-02', '2016-06-03',
        '2016-06-06', '2016-06-07', '2016-06-08'
    ],
    'Settings': [len(total_groups_June01_24p)] * 6,
    'Consultas': [
        len(total_groups_June01_24p['setting 1']),
        len(total_groups_June02_24p['setting 1']),
        len(total_groups_June03_24p['setting 1']),
        len(total_groups_June06_24p['setting 1']),
        len(total_groups_June07_24p['setting 1']),
        len(total_groups_June08_24p['setting 1']),
    ],
    'Idle Cost': [
        average_costs_total_June01_R11['average_idle_cost'],
        average_costs_total_June02_R11['average_idle_cost'],
        average_costs_total_June03_R11['average_idle_cost'],
        average_costs_total_June06_R11['average_idle_cost'],
        average_costs_total_June07_R11['average_idle_cost'],
        average_costs_total_June08_R11['average_idle_cost']
    ],
    'Waiting Cost': [
        average_costs_total_June01_R11['average_waiting_cost'],
        average_costs_total_June02_R11['average_waiting_cost'],
        average_costs_total_June03_R11['average_waiting_cost'],
        average_costs_total_June06_R11['average_waiting_cost'],
        average_costs_total_June07_R11['average_waiting_cost'],
        average_costs_total_June08_R11['average_waiting_cost']
    ],
    'Overbooking Cost': [
        average_costs_total_June01_R11['average_overtime_cost'],
        average_costs_total_June02_R11['average_overtime_cost'],
        average_costs_total_June03_R11['average_overtime_cost'],
        average_costs_total_June06_R11['average_overtime_cost'],
        average_costs_total_June07_R11['average_overtime_cost'],
        average_costs_total_June08_R11['average_overtime_cost']
    ]
}

tabla_resumen_R11 = pd.DataFrame(data_R11)

exportar_tabla_resumen_a_excel(tabla_resumen_R11, 'R11')

# Mostrar el DataFrame
tabla_resumen_R11

El DataFrame ha sido exportado exitosamente a Costes_promedio_R11.xlsx.


,Fecha,Settings,Consultas,Idle Cost,Waiting Cost,Overbooking Cost
0,2016-06-01,10,127,4.502362,0.0,0.0
1,2016-06-02,10,135,4.388148,0.0,0.0
2,2016-06-03,10,131,4.704580,0.0,0.0
3,2016-06-06,10,158,4.492405,0.0,0.0
4,2016-06-07,10,161,4.227950,0.0,0.0
5,2016-06-08,10,171,4.115205,0.0,0.0


## 2.2 Regla de Asignación de Cita
### **1 Slot : 2 Patients -> 3 Slots : 1 Patient -> 1 Slot : 2 Patients ...**

24 Slots - 30 pacientes citados

### 2.2.1 Asignaciones de cita

In [18]:
NUM_SLOTS_PER_DAY = 24

# 01 Junio 2016:
# 10 settings x 102 consultas = 1020 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June01_R1231 = {}
for i, setting in enumerate(total_groups_June01_30p.keys()):
    for j, group in enumerate(total_groups_June01_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June01_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June01_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June01_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 02 Junio 2016:
# 10 settings x 108 consultas = 1080 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June02_R1231 = {}
for i, setting in enumerate(total_groups_June02_30p.keys()):
    for j, group in enumerate(total_groups_June02_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June02_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June02_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June02_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 03 Junio 2016:
# 10 settings x 105 consultas = 1050 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June03_R1231 = {}
for i, setting in enumerate(total_groups_June03_30p.keys()):
    for j, group in enumerate(total_groups_June03_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June03_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June03_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June03_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 06 Junio 2016:
# 10 settings x 126 consultas = 1260 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June06_R1231 = {}
for i, setting in enumerate(total_groups_June06_30p.keys()):
    for j, group in enumerate(total_groups_June06_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June06_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June06_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June06_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 07 Junio 2016:
# 10 settings x 129 consultas = 1290 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June07_R1231 = {}
for i, setting in enumerate(total_groups_June07_30p.keys()):
    for j, group in enumerate(total_groups_June07_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June07_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June07_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June07_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 08 Junio 2016:
# 10 settings x 137 consultas = 1370 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June08_R1231 = {}
for i, setting in enumerate(total_groups_June08_30p.keys()):
    for j, group in enumerate(total_groups_June08_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June08_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June08_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June08_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

### 2.2.2 Cálculo de Costes: Idle Time, Waiting Time y Over Time

In [19]:
print('\033[1mHIPÓTESIS: Regla de Asignación de Cita\n \
\t\t 1 Slot : 2 Patients -> 3 Slots : 1 Patient -> 1 Slot : 2 Patients ...\033[0m\n')

# 1 Junio 2016
print('\033[1m01 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June01_R1231, average_costs_total_June01_R1231 = calculo_costes(
    set = total_set_June01,
    groups = total_groups_June01_30p,
    slot_apps = slot_apps_total_June01_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 2 Junio 2016
print('\033[1m02 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June02_R1231, average_costs_total_June02_R1231 = calculo_costes(
    set = total_set_June02,
    groups = total_groups_June02_30p,
    slot_apps = slot_apps_total_June02_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 3 Junio 2016
print('\033[1m03 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June03_R1231, average_costs_total_June03_R1231 = calculo_costes(
    set = total_set_June03,
    groups = total_groups_June03_30p,
    slot_apps = slot_apps_total_June03_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 6 Junio 2016
print('\033[1m06 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June06_R1231, average_costs_total_June06_R1231 = calculo_costes(
    set = total_set_June06,
    groups = total_groups_June06_30p,
    slot_apps = slot_apps_total_June06_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 7 Junio 2016
print('\033[1m07 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June07_R1231, average_costs_total_June07_R1231 = calculo_costes(
    set = total_set_June07,
    groups = total_groups_June07_30p,
    slot_apps = slot_apps_total_June07_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 8 Junio 2016
print('\033[1m08 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June08_R1231, average_costs_total_June08_R1231 = calculo_costes(
    set = total_set_June08,
    groups = total_groups_June08_30p,
    slot_apps = slot_apps_total_June08_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

HIPÓTESIS: Regla de Asignación de Cita
 		 1 Slot : 2 Patients -> 3 Slots : 1 Patient -> 1 Slot : 2 Patients ...

01 de junio:
-----------
Set Total con todas las Citas Médicas:
Número de consultas: 1020  =  10 settings diferentes * 102 doctores pasando consulta.
La media de los Costes de Inactividad en los 10 escenarios distintos para las consultas de los 102 doctores distintos es de 2.00.
La media de los Costes de Espera de los Pacientes en los 10 escenarios distintos para las consultas de los 102 doctores distintos es de 21.16.
La media de los Costes de Tiempo Extra en los 10 escenarios distintos para las consultas de los 102 doctores distintos es de 2.37.
-------------------------------------------------------------------------------------------------------------------------------------------

02 de junio:
-----------
Set Total con todas las Citas Médicas:
Número de consultas: 1080  =  10 settings diferentes * 108 doctores pasando consulta.
La media de los Costes de Inactividad en 

In [20]:
# Crear la Tabla RESUMEN con los costes medios para esta Hipótesis R1231
data_R1231 = {
    'Fecha': [
        '2016-06-01', '2016-06-02', '2016-06-03',
        '2016-06-06', '2016-06-07', '2016-06-08'
    ],
    'Settings': [len(total_groups_June01_30p)] * 6,
    'Consultas': [
        len(total_groups_June01_30p['setting 1']),
        len(total_groups_June02_30p['setting 1']),
        len(total_groups_June03_30p['setting 1']),
        len(total_groups_June06_30p['setting 1']),
        len(total_groups_June07_30p['setting 1']),
        len(total_groups_June08_30p['setting 1']),
    ],
    'Idle Cost': [
        average_costs_total_June01_R1231['average_idle_cost'],
        average_costs_total_June02_R1231['average_idle_cost'],
        average_costs_total_June03_R1231['average_idle_cost'],
        average_costs_total_June06_R1231['average_idle_cost'],
        average_costs_total_June07_R1231['average_idle_cost'],
        average_costs_total_June08_R1231['average_idle_cost']
    ],
    'Waiting Cost': [
        average_costs_total_June01_R1231['average_waiting_cost'],
        average_costs_total_June02_R1231['average_waiting_cost'],
        average_costs_total_June03_R1231['average_waiting_cost'],
        average_costs_total_June06_R1231['average_waiting_cost'],
        average_costs_total_June07_R1231['average_waiting_cost'],
        average_costs_total_June08_R1231['average_waiting_cost']
    ],
    'Overbooking Cost': [
        average_costs_total_June01_R1231['average_overtime_cost'],
        average_costs_total_June02_R1231['average_overtime_cost'],
        average_costs_total_June03_R1231['average_overtime_cost'],
        average_costs_total_June06_R1231['average_overtime_cost'],
        average_costs_total_June07_R1231['average_overtime_cost'],
        average_costs_total_June08_R1231['average_overtime_cost']
    ]
}

tabla_resumen_R1231 = pd.DataFrame(data_R1231)

exportar_tabla_resumen_a_excel(tabla_resumen_R1231, 'R1231')

# Mostrar el DataFrame
tabla_resumen_R1231


El DataFrame ha sido exportado exitosamente a Costes_promedio_R1231.xlsx.


,Fecha,Settings,Consultas,Idle Cost,Waiting Cost,Overbooking Cost
0,2016-06-01,10,102,1.995098,21.155882,2.369608
1,2016-06-02,10,108,1.916667,21.423148,2.431481
2,2016-06-03,10,105,2.045714,19.973333,2.163810
3,2016-06-06,10,126,2.061111,21.428571,2.445238
4,2016-06-07,10,129,1.727132,22.693023,2.436434
5,2016-06-08,10,137,1.568613,22.103650,2.427007


## 2.3 Regla de Asignación de Cita
### **1 Slot : 2 Patients -> 2 Slots : 1 Patient (sólo 3 veces en ambos turnos)**

24 Slots - 30 pacientes citados

### 2.3.1 Asignaciones de cita

In [24]:
NUM_SLOTS_PER_DAY = 24

# 01 Junio 2016:
# 10 settings x 102 consultas = 1020 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June01_R1221 = {}
for i, setting in enumerate(total_groups_June01_30p.keys()):
    for j, group in enumerate(total_groups_June01_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June01_R1221[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 4, 7, 13, 16, 19]:
                slot_apps_total_June01_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June01_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 02 Junio 2016:
# 10 settings x 108 consultas = 1080 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June02_R1221 = {}
for i, setting in enumerate(total_groups_June02_30p.keys()):
    for j, group in enumerate(total_groups_June02_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June02_R1221[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 4, 7, 13, 16, 19]:
                slot_apps_total_June02_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June02_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 03 Junio 2016:
# 10 settings x 105 consultas = 1050 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June03_R1221 = {}
for i, setting in enumerate(total_groups_June03_30p.keys()):
    for j, group in enumerate(total_groups_June03_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June03_R1221[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 4, 7, 13, 16, 19]:
                slot_apps_total_June03_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June03_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 06 Junio 2016:
# 10 settings x 126 consultas = 1260 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June06_R1221 = {}
for i, setting in enumerate(total_groups_June06_30p.keys()):
    for j, group in enumerate(total_groups_June06_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June06_R1221[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 4, 7, 13, 16, 19]:
                slot_apps_total_June06_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June06_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 07 Junio 2016:
# 10 settings x 129 consultas = 1290 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June07_R1221 = {}
for i, setting in enumerate(total_groups_June07_30p.keys()):
    for j, group in enumerate(total_groups_June07_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June07_R1221[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 4, 7, 13, 16, 19]:
                slot_apps_total_June07_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June07_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 08 Junio 2016:
# 10 settings x 137 consultas = 1370 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June08_R1221 = {}
for i, setting in enumerate(total_groups_June08_30p.keys()):
    for j, group in enumerate(total_groups_June08_30p[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June08_R1221[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 4, 7, 13, 16, 19]:
                slot_apps_total_June08_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June08_R1221[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

In [25]:
print('\033[1mHIPÓTESIS: Regla de Asignación de Cita\n \
\t\t 1 Slot : 2 Patients -> 2 Slots : 1 Patient (sólo 3 veces en ambos turnos)\033[0m\n')

# 1 Junio 2016
print('\033[1m01 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June01_R1221, average_costs_total_June01_R1221 = calculo_costes(
    set = total_set_June01,
    groups = total_groups_June01_30p,
    slot_apps = slot_apps_total_June01_R1221
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 2 Junio 2016
print('\033[1m02 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June02_R1221, average_costs_total_June02_R1221 = calculo_costes(
    set = total_set_June02,
    groups = total_groups_June02_30p,
    slot_apps = slot_apps_total_June02_R1221
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 3 Junio 2016
print('\033[1m03 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June03_R1221, average_costs_total_June03_R1221 = calculo_costes(
    set = total_set_June03,
    groups = total_groups_June03_30p,
    slot_apps = slot_apps_total_June03_R1221
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 6 Junio 2016
print('\033[1m06 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June06_R1221, average_costs_total_June06_R1221 = calculo_costes(
    set = total_set_June06,
    groups = total_groups_June06_30p,
    slot_apps = slot_apps_total_June06_R1221
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 7 Junio 2016
print('\033[1m07 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June07_R1221, average_costs_total_June07_R1221 = calculo_costes(
    set = total_set_June07,
    groups = total_groups_June07_30p,
    slot_apps = slot_apps_total_June07_R1221
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 8 Junio 2016
print('\033[1m08 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June08_R1221, average_costs_total_June08_R1221 = calculo_costes(
    set = total_set_June08,
    groups = total_groups_June08_30p,
    slot_apps = slot_apps_total_June08_R1221
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

HIPÓTESIS: Regla de Asignación de Cita
 		 1 Slot : 2 Patients -> 2 Slots : 1 Patient (sólo 3 veces en ambos turnos)

01 de junio:
-----------
Set Total con todas las Citas Médicas:
Número de consultas: 1020  =  10 settings diferentes * 102 doctores pasando consulta.
La media de los Costes de Inactividad en los 10 escenarios distintos para las consultas de los 102 doctores distintos es de 1.81.
La media de los Costes de Espera de los Pacientes en los 10 escenarios distintos para las consultas de los 102 doctores distintos es de 23.97.
La media de los Costes de Tiempo Extra en los 10 escenarios distintos para las consultas de los 102 doctores distintos es de 2.18.
-------------------------------------------------------------------------------------------------------------------------------------------

02 de junio:
-----------
Set Total con todas las Citas Médicas:
Número de consultas: 1080  =  10 settings diferentes * 108 doctores pasando consulta.
La media de los Costes de Inactividad

In [26]:
# Crear la Tabla RESUMEN con los costes medios para esta Hipótesis R1231
data_R1221 = {
    'Fecha': [
        '2016-06-01', '2016-06-02', '2016-06-03',
        '2016-06-06', '2016-06-07', '2016-06-08'
    ],
    'Settings': [len(total_groups_June01_30p)] * 6,
    'Consultas': [
        len(total_groups_June01_30p['setting 1']),
        len(total_groups_June02_30p['setting 1']),
        len(total_groups_June03_30p['setting 1']),
        len(total_groups_June06_30p['setting 1']),
        len(total_groups_June07_30p['setting 1']),
        len(total_groups_June08_30p['setting 1']),
    ],
    'Idle Cost': [
        average_costs_total_June01_R1221['average_idle_cost'],
        average_costs_total_June02_R1221['average_idle_cost'],
        average_costs_total_June03_R1221['average_idle_cost'],
        average_costs_total_June06_R1221['average_idle_cost'],
        average_costs_total_June07_R1221['average_idle_cost'],
        average_costs_total_June08_R1221['average_idle_cost']
    ],
    'Waiting Cost': [
        average_costs_total_June01_R1221['average_waiting_cost'],
        average_costs_total_June02_R1221['average_waiting_cost'],
        average_costs_total_June03_R1221['average_waiting_cost'],
        average_costs_total_June06_R1221['average_waiting_cost'],
        average_costs_total_June07_R1221['average_waiting_cost'],
        average_costs_total_June08_R1221['average_waiting_cost']
    ],
    'Overbooking Cost': [
        average_costs_total_June01_R1221['average_overtime_cost'],
        average_costs_total_June02_R1221['average_overtime_cost'],
        average_costs_total_June03_R1221['average_overtime_cost'],
        average_costs_total_June06_R1221['average_overtime_cost'],
        average_costs_total_June07_R1221['average_overtime_cost'],
        average_costs_total_June08_R1221['average_overtime_cost']
    ]
}

tabla_resumen_R1221 = pd.DataFrame(data_R1221)

exportar_tabla_resumen_a_excel(tabla_resumen_R1221, 'R1221')

# Mostrar el DataFrame
tabla_resumen_R1221


El DataFrame ha sido exportado exitosamente a Costes_promedio_R1221.xlsx.


,Fecha,Settings,Consultas,Idle Cost,Waiting Cost,Overbooking Cost
0,2016-06-01,10,102,1.809804,23.966667,2.184314
1,2016-06-02,10,108,1.704630,24.137963,2.219444
2,2016-06-03,10,105,1.871429,22.749524,1.989524
3,2016-06-06,10,126,1.884127,24.271429,2.268254
4,2016-06-07,10,129,1.586047,25.900000,2.295349
5,2016-06-08,10,137,1.394161,25.080292,2.252555
